# PROJECT BIG DATA: GROUP 19: CLIMATE CHANGE

In [ ]:
# import all the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import sklearn
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
# read all csv into df
df = pd.read_csv("data/owid-co2-data.csv")
# df = df[(df['year']>=2000 )&(df['year']<=2019 ) ]
df = df[(df['year']>=2000)]
df.shape
# external resource
energy = pd.read_csv("data/owid-energy-data.csv")
energy.shape
# energy = energy [(energy['year']>= 2000)& (energy['year']<=2019)]
df_country= pd.read_csv("data/GlobalLandTemperaturesByCountry.csv")


## 📑 Table of Contents

- [EDA](#eda)
  - [Data wrangling:](#data-wrangling)
    - [Quality checks:](#quality-checks)
    - [Clean the data](#clean-the-data)
- [Machine Learning models: For 3 main research questions:](#machine-learning-modaels-for-3-main-research-questions)
  - [ML: RQ: How accurately can a machine-learning model predict a country’s per-capita CO₂ emissions…?](#ml-rq-how-accurately-can-a-machine-learning-model-predict-a-countrys-per-capita-co₂-emissions)
    - [Preprocessing data](#preprocessing-data)
    - [Scaling the data by normalization](#scaling-the-data-by-normalization)
    - [Ridge Regression: (With hypertuning to find α)](#ridge-regression-with-hypertuning-to-find-α)
    - [Random Forest (Hypertuning parameters)](#random-forest-hypertuning-parameters)
    - [XGB Regressor + Hypertuning parameters](#xgb-regressor--hypertuning-parameters)
    - [Gradient Boosting Regressor + Hypertuning parameters](#gradient-boosting-regressor--hypertuning-parameters)
    - [K-Fold Cross-Validation for all models](#k-fold-cross-validation-for-all-models)
  - [ML: RQ: How significant is the relationship between latitude and temperature and can latitude be used to accurately predict temperatures?](#ml-rq-how-significant-is-the-relationship-between-latitude-and-temperature-and-can-latitude-be-used-to-accurately-predict-temperatures)
  - [Visualizing temperature trends](#visualizing-temperature-trends)
  - [Correlation matrix](#correlation-matrix)
  - [Regression models - latitude only](#regression-models-latitude-only)
  - [Regression models - laitutde + longitude](#regression-models-latitude-longitude)
  - [Scatter plot and Hexbin - Actual vs Predicted](#Scatter-plot-Hexbin-Actuals-Predicted)

# EDA

In [ ]:
df_city = pd.read_csv("data/GlobalLandTemperaturesByCity.csv")
df_city['Date'] = pd.to_datetime(df_city['dt'], errors='coerce')
df_city['Year'] = df_city['Date'].dt.year
df_city = df_city.dropna(subset=['Date', 'AverageTemperature', 'Latitude', 'Longitude'])

lat_values = df_city['Latitude'].str.extract(r'(\d+\.?\d*)')[0].astype(float)
is_south = df_city['Latitude'].str.endswith('S')
df_city['Latitude'] = lat_values * np.where(is_south, -1, 1)

long_values = df_city['Longitude'].str.extract(r'(\d+\.?\d*)')[0].astype(float)
is_west = df_city['Longitude'].str.endswith('W')
df_city['Longitude'] = long_values * np.where(is_west, -1, 1)

df_city = df_city[df_city['Year'] >= 1875]
df_city['Latitude_bin'] = pd.cut(df_city['Latitude'], bins=np.arange(-90, 100, 10))

print(df_city.head(3))
print(df_city.tail(3))

### Data wrangling:

In [ ]:
df_country.head(3)

In [ ]:
df_country.tail(3)


In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
energy.head(3)

In [ ]:
energy.tail(3)

In [ ]:
print(df_country.describe(include = "all"))
print(df.describe(include = 'all'))
print(energy.describe(include = 'all'))

#### Quality checks:

In [ ]:
print(df_country.shape)
print("Total countries:", df_country['Country'].nunique())
print("Total months:", df_country['dt'].nunique())

print(df.shape)
print(energy.shape)

 `drop records`** reasonable to restrict the analysis to data from **1900 onward**(significantly reduces the sparsity and improves data reliability.


In [ ]:
df_country['dt'] = pd.to_datetime(df_country['dt'])
df_country_clean = df_country[df_country['dt'].dt.year >=1900]
df_country_clean

#### Clean the data

In [ ]:
df = df.dropna(axis = 1,
                   thresh=int(len(df) * 0.3)  
)


In [ ]:

energy = energy.dropna(subset = ['iso_code'])
energy['iso_code'].isnull().sum()


In [ ]:
energy[['renewables_consumption', 'primary_energy_consumption']].corr(method='pearson')


### EDA: `df_country`:

In [ ]:
### a general function: choropleth map: better to see whether the data is well-cleaning or not.
def plot_country_choropleth(df, country_col='Country', value_col=None, aggfunc='count',title='Choropleth Map by Country', color_scale='viridis'):
    """
    Creates a choropleth map using Plotly based on country-level data.
    """
    if value_col:
        df_agg = df.groupby(country_col)[value_col].agg(aggfunc).reset_index()
        df_agg.columns = [country_col, 'Value']
    else:
        df_agg = df[country_col].value_counts().reset_index()
        df_agg.columns = [country_col, 'Value']

    fig = px.choropleth(df_agg, locations=country_col, locationmode='country names', color='Value',
        color_continuous_scale=color_scale, title=title
    )
    fig.update_layout(coloraxis_colorbar=dict(title='Value'),
        margin=dict(l=0, r=0, t=50, b=0))
    fig.show()

    # plot_country_choropleth(df_country,
                        # title='Number of Temperature Records by Country(All RECORDS)')

In [ ]:
df_country['dt'] = pd.to_datetime(df_country['dt'])
df_country_clean = df_country[df_country['dt'].dt.year >=1900]
df_country_clean

In [ ]:
df_country_clean.isnull().sum()

Because this is a time-series dataset, filling the < 1 % temperature gaps with an overall mean

=> Safely remove the rows with missing value

In [ ]:
df_country_clean = df_country_clean.dropna(subset=['AverageTemperature'])
# a choropleth map to show that the data are well-cleaning:
# plot_country_choropleth(df_country_clean,
#                         title='Number of Temperature Records by Country (from 1900)')

In [ ]:
plot_country_choropleth(df_country_clean,
                        value_col= 'AverageTemperature',
                        aggfunc = 'mean',
                        title = 'Average Temperature of Countries')

#### RQ: What are the countries with minimum and maximum average temperature?

In [ ]:
df_country_clean[df_country_clean['AverageTemperature']== df_country_clean['AverageTemperature'].min()]
df_country_clean[df_country_clean['AverageTemperature']== df_country_clean['AverageTemperature'].max()]

In [ ]:
def plot_kde_temperature_comparison(df, countries, year):
    """
    Plots KDE curves of average temperature for multiple countries in a given year.
    Parameters:
    - df: DataFrame with 'Country', 'dt', 'AverageTemperature'
    - countries: list of country names, e.g. ['Kuwait', 'Greenland']
    - year: int, e.g. 2012
    """
    df = df[df['Country'].isin(countries)].copy()
    df.dropna(subset=['AverageTemperature', 'dt'], inplace=True)
    df['dt'] = pd.to_datetime(df['dt'])
    df['year'] = df['dt'].dt.year
    df = df[df['year'] == year]
    custom_palette = {'Greenland': 'skyblue', 'Kuwait': 'orange'}

    plt.figure(figsize=(10, 5))
    sns.kdeplot(data=df,
        x='AverageTemperature',
        hue='Country',
        palette=custom_palette,
        fill=True,
        common_norm=False,
        alpha=0.5,
        linewidth=2
    )
    plt.title(f'KDE of Average Temperature in {", ".join(countries)} ({year})')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Density')
    plt.tight_layout()
    plt.show()
    plot_kde_temperature_comparison(df_country_clean, ['Greenland', 'Kuwait'], 2012)


#### RQ: How has the average yearly temperature changed from 1900 to 2020?

In [ ]:
df_country_clean['Year'] = df_country_clean['dt'].dt.year
df_yearly_avg = df_country_clean.groupby('Year')['AverageTemperature'].mean().reset_index()

sns.lmplot(data=df_yearly_avg, x='Year', y='AverageTemperature', palette= 'viridis')
sns.set_style("whitegrid")
plt.title('Yearly Average Temperature')
plt.xlabel('Year')
plt.ylabel('Average Temperature (Celsius)')
plt.show()

The regression line in the plot **highlights a significant upward trend**, suggesting that the average temperature has steadily increased over the last century, consistent with **global warming patterns.**

### EDA for `df`(emissions):

In [ ]:
df

In [ ]:
# get rid of sparse
df_emissions = df[df['year']>=1900]
df_emissions.isnull().sum()


Handling the missing values: drop the columns contain >= 30% missing data points

In [ ]:
df_emissions = df_emissions.dropna(
    axis=1,
    thresh=int(len(df_emissions) * 0.3)  
)
print(df_emissions.shape)
print(df_emissions.columns)


#### RQ: The correlation of co2 with economics and energy factors

In [ ]:
sns.scatterplot(data=df_emissions[['gdp','co2']].dropna(), x='gdp', y='co2', alpha=0.6)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('GDP (log scale)')
plt.ylabel('CO2 emissions (log scale)')
plt.title('GDP vs. CO2 emissions')
plt.tight_layout()
plt.show()

In [ ]:
sns.scatterplot(data=df_emissions, x='energy_per_capita', y='co2_per_capita', alpha=0.6)
plt.xscale('log'); plt.yscale('log')
plt.title('Energy vs CO2 per capita (log–log)')
plt.show()

In [ ]:
df_emissions['gdp_per_capita'] = df_emissions['gdp'] / df_emissions['population']

df_emissions['gdp_per_capita'] = df_emissions['gdp_per_capita'].round(3) 


df_emissions

In [ ]:
sns.scatterplot(data=df_emissions, x='gdp_per_capita', y='co2_per_capita', alpha = 0.6)
plt.xscale('log'); plt.yscale('log')
plt.title('GDP vs CO2 per capita (log–log)')
plt.show()

#### RQ: Which regions or economic blocs were the top $CO_2$ emitters in the most recent year, based on OWID aggregate groups?

In [ ]:
def top_emitters(df,start, end, level,  metric="co2", top_n=5):
    """
    Return the top-N emitters for a given period.

    Parameters
    ----------
    df : pandas.DataFrame
        OWID / GCP emissions table. Must contain columns
        'year', 'country', 'iso_code', and the metric column.
    start_year, end_year : int
        Inclusive year boundaries (e.g. 2013, 2023).
    level : {"aggregate", "country"}
        aggregate – OWID/GCP group rows (iso_code is missing OR starts 'OWID_').
        country   – ISO-3166 alpha-3 rows only (len == 3, all A-Z).
    metric : str, default "co2"
        Column to sum.
    top_n : int, default 5
        How many rows to return.
    """

    df = df.copy()
    df.loc[df["iso_code"] == "", "iso_code"] = np.nan

    recent = df[(df["year"] >= start) & (df["year"] <= end)]

    
    if level == "country":
        mask = (recent["iso_code"].notna()
            & recent["iso_code"].str.fullmatch(r"[A-Z]{3}")
        )
    elif level == "aggregate":
        mask = pd.Series(True, index=recent.index)
    else:
        raise ValueError("level must be 'aggregate' or 'country'")

    subset = recent[mask]

    cumulative = (subset.groupby("country")[metric].sum()
        .sort_values(ascending=False)
        .head(top_n)
        .reset_index()
        .rename(columns={metric: f"cumulative_{metric}_{start}_{end}"}) )

    return cumulative

In [ ]:
agg_top5 = top_emitters(df_emissions, 2013, 2023, level="aggregate")
print(agg_top5)
fig = px.bar(agg_top5, x="country", y="cumulative_co2_2013_2023", color="country",  
    text="cumulative_co2_2013_2023",  
    title="Cumulative CO2 Emissions (2013–2023) by Aggregation",
    labels={"cumulative_co2_2013_2023": "Cumulative CO2 (Mt)", "country": "Aggregation"},
)
fig.update_traces(texttemplate='%{text:.2f}')
fig.show()

#### RQ: Which countries are the top emitters in the recent ten years? (2013-2023)

In [ ]:
# Top 5 individual countries, 2013-2023
country_top5 = top_emitters(df_emissions, 2013, 2023, level="country")
print(country_top5)
fig = px.bar(
    country_top5,  
    x="country",
    y="cumulative_co2_2013_2023",
    color="country",  
    text="cumulative_co2_2013_2023",  
    title="Cumulative CO₂ Emissions (2013–2023) by Country",
    labels={"cumulative_co2_2013_2023": "Cumulative CO2 (Mt)", "country": "Country"},
)
fig.update_traces(texttemplate='%{text:.2f}')
fig.show()

#### Temperature change from $CO_2$ by continents from 1950 to 2020:


In [ ]:
# Filter only continent-level data
continents = ["Africa", "Asia", "Europe", "North America", "South America", "Oceania"]
df_continents = df_emissions[
    df_emissions["country"].isin(continents) &
    df_emissions["temperature_change_from_co2"].notna()
]

# Optional: recent years only
df_continents = df_continents[df_continents["year"] >= 1950]

# Plot
fig = px.line(df_continents, x="year", y="temperature_change_from_co2", color="country",
              title="Temperature Change from CO₂ by Continent (1950–present)",
              labels={"country": "Continent", "temperature_change_from_co2": "Temp Change (°C)"})
fig.show()

#### Average Temperature and $CO_2$ emissions through years

In [ ]:
# 1. Ensure 'dt' is datetime
df_country_clean["dt"] = pd.to_datetime(df_country_clean["dt"])

# 2. Extract year
df_country_clean["year"] = df_country_clean["dt"].dt.year

# 3. Group by year and average both temperature and uncertainty
df_yearly = df_country_clean.groupby("year")[["AverageTemperature", "AverageTemperatureUncertainty"]].mean().reset_index()

# 4. Plot with confidence band
plt.figure(figsize=(15, 8))
sns.lineplot(
    x="year",
    y="AverageTemperature",
    data=df_yearly,
    label="Avg Temperature"
)
plt.fill_between(
    df_yearly["year"],
    df_yearly["AverageTemperature"] - df_yearly["AverageTemperatureUncertainty"],
    df_yearly["AverageTemperature"] + df_yearly["AverageTemperatureUncertainty"],
    alpha=0.3,
    label="+- Uncertainty"
)

plt.title("Average Temperature through Years")
plt.xlabel("Year")
plt.ylabel("Average Temperature (°C)")
plt.legend()
plt.show()

In [ ]:
### CO2 emissions trends over the years
# Keep only the country no more aggregation
plt.figure(figsize = (15,8))
sns.lineplot(x="year", y="co2", data=df_emissions)
plt.title('CO2 emission trends over the years')
plt.show()

#### $CO_2$ emissions over time of top 5 emitters countries since 1990

In [ ]:
selected_countries = ["China","Russia", "United States", "Japan", "India"]
df_filtered = df_emissions[
    (df_emissions["country"].isin(selected_countries)) &
    (df_emissions["co2"].notna())
]
fig = px.box(
    df_filtered[df_filtered["year"] >= 1990],
    x="country",
    y="co2",
    log_y=True,                 # compresses huge differences
    points='outliers',          # show only true outliers
    title="CO₂ Emissions Since 1990 (log scale)",
    labels={"co2": "CO₂ Emissions (million tonnes, log scale)"}
)
fig.show()

#### CO2 emission per capita trends over the years

In [ ]:
plt.figure(figsize = (15,8))
sns.lineplot(x="year", y="co2_per_capita", data=df)
plt.title('CO2 emission per capita trends')
plt.show()

## SDA: 

### Has the average annual temperature significantly increased when comparing the early 20th century to the early 21st century
- Testing hypothesis: 

    $H_0$: $\mu_{\text{early}} = \mu_{\text{recent}}$

    $H_a$: $\mu_{\text{early}} \ne \mu_{\text{recent}}$

#### Diagnostics plots and tests:


In [ ]:
# Residuals vs Fitted plot
import statsmodels.api as sm
from scipy.stats import linregress

result = linregress(df_yearly_avg['Year'], df_yearly_avg['AverageTemperature'])
print(result)
# Fit model using statsmodels
X = df_yearly_avg['Year']
y = df_yearly_avg['AverageTemperature']
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()

# 1. Residuals vs Fitted
residuals = model.resid
fitted = model.fittedvalues
sns.residplot(x=fitted, y=residuals, lowess=True)
plt.axhline(0, linestyle='--')
plt.title('Residuals vs. Fitted')
plt.show()


In [ ]:
# Durbin-Watson test
from statsmodels.stats.stattools import durbin_watson
dw_stat = durbin_watson(model.resid)
print(f"Durbin-Watson statistic: {dw_stat:.3f}")

In [ ]:
# 2. Normal Q-Q Plot
sm.qqplot(residuals, line='s')
plt.title('Normal Q-Q')
plt.show()

In [ ]:
# Select relevant columns and drop NaNs
df_temp = df_yearly_avg[['Year', 'AverageTemperature']].dropna()

# Define periods
early = df_temp[(df_temp['Year'] >= 1900) & (df_temp['Year'] <= 1920)]['AverageTemperature']
recent = df_temp[(df_temp['Year'] >= 2000) & (df_temp['Year'] <= 2020)]['AverageTemperature']

# Create Q-Q plots side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sm.qqplot(early, line='s', ax=axes[0])
axes[0].set_title('Q-Q Plot: 1900–1920')

sm.qqplot(recent, line='s', ax=axes[1])
axes[1].set_title('Q-Q Plot: 2000–2020')

plt.tight_layout()
plt.show()

#### Testing process

In [ ]:
from scipy.stats import ttest_ind
# 2-sample t-tests
early = df_yearly_avg[df_yearly_avg['Year'] < 1950]['AverageTemperature']
recent = df_yearly_avg[df_yearly_avg['Year'] >= 1950]['AverageTemperature']

t_stat, p_value = ttest_ind(early, recent, equal_var=False)  
print(f"t = {t_stat:.3f}, p-value = {p_value:.4e}")

In [ ]:
early = df_yearly_avg.query("1900 <= Year <= 1920").copy()
recent = df_yearly_avg.query("2000 <= Year <= 2020").copy()

early["Period"] = "1900–1920"
recent["Period"] = "2000–2020"
plot_df = pd.concat([early, recent], ignore_index=True)
sns.kdeplot(data=plot_df, x="AverageTemperature", hue="Period", fill=True, common_norm=False)

import scipy


early   = df_yearly_avg.query("Year.between(1900, 1920)")["AverageTemperature"]
recent  = df_yearly_avg.query("Year.between(2000, 2020)")["AverageTemperature"]
# Normality checks
for label, series in {"Early": early, "Recent": recent}.items():
    stat, p = scipy.stats.shapiro(series)
    print(f"{label} Shapiro p = {p:.3f}")

# Variance equality
levene_p = scipy.stats.levene(early, recent).pvalue
print(f"Levene p = {levene_p:.3f}")

# Welch two-sample t-test
t_stat, p_val = scipy.stats.ttest_ind(early, recent, equal_var=False)
print(f"Welch t = {t_stat:.2f}, p = {p_val:.3e}")


### SDA: Monika

## Machine Learning models: For 3 main research questions: 
- (Duncan)
- (Oscar)
- How accurately can a machine-learning model predict a country’s total CO₂ emissions from its share of renewable-energy consumption (and other socio-economic indicators)?

###  ML: RQ: How accurately can a machine-learning model predict a country’s total CO₂ emissions from its share of renewable-energy consumption (and other socio-economic indicators)?


#### Preprocessing data

In [ ]:
# Keep only the country no more aggregation
df = df.dropna(subset=['iso_code'])
# indicators = ['gdp', 'population', 'primary_energy_consumption','renewables_consumption', 'oil_consumption' ]
keys = ['country', 'year']
df_sub     = df[ keys + ['gdp', 'population', 'co2','primary_energy_consumption' ]]
energy_sub = energy[ keys + ['renewables_consumption','oil_consumption'] ]

merged = pd.merge(df_sub, energy_sub, on=keys, how='inner')
# Cook's distance
influential_countries = ['China', 'India', 'United States']

# Filter out these countries
merged = merged[~merged['country'].isin(influential_countries)]

merged


In [ ]:
#Drop sparse countries
core = ['gdp','population','primary_energy_consumption','co2']
coverage = (
    merged.assign(nonmiss=lambda d: d[core].notnull().all(axis=1))
      .groupby('country')['nonmiss'].mean()
)
keep = coverage[coverage >= 0.6].index
merged = merged[merged['country'].isin(keep)].copy()
# Imputation for other missing records per country
cols_to_impute = ['gdp', 'primary_energy_consumption','oil_consumption', 'renewables_consumption']
# 3. Per-country median imputation
merged[cols_to_impute] = merged.groupby('country')[cols_to_impute] \
                       .transform(lambda x: x.fillna(x.median()))
# fill 0 for oil_consumption, and renewable
merged['renewables_consumption'] = merged['renewables_consumption'].fillna(0)
merged['oil_consumption'] = merged['oil_consumption'].fillna(0)


In [ ]:
merged.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
merged['year'] = merged['year'].astype(int)
prediction_features = ['year', 'gdp','population','primary_energy_consumption','renewables_consumption','oil_consumption']
target = 'co2'
X = merged[prediction_features]
y = merged[target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=False
)
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print("\nFeature columns:", X_train.columns.tolist())


#### Scaling the data by normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Ridge Regression: (With hypertuning to find $\alpha$)

In [ ]:
### Finding alpha 
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
alphas = np.linspace(0.01, 10, 50) 
ridge_cv = RidgeCV(alphas=alphas, scoring='neg_root_mean_squared_error', cv=5)
ridge_cv.fit(X_train_scaled, y_train)

print("Best alpha:", ridge_cv.alpha_)

In [ ]:
coefs = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    coefs.append(ridge.coef_)

coefs = np.array(coefs)

plt.figure(figsize=(10, 6))
for i in range(coefs.shape[1]):
    plt.plot(alphas, coefs[:, i], label=X_train.columns[i])

plt.xlabel("Lambda")
plt.ylabel("Coefficient Value")
plt.title("Ridge Coefficients vs. Regularization Strength (lambda)")
plt.legend(loc='best')
plt.grid(True)
plt.show()


In [ ]:
### Ridge:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import root_mean_squared_error
#Model
ridge = Ridge(alpha = 0.01 )
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)
# Metrics performance
rmse = root_mean_squared_error(y_test, y_pred_ridge)
r2 = r2_score(y_test, y_pred_ridge)
mae = mean_absolute_error(y_test, y_pred_ridge)
print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

In [ ]:
#  1) Extract absolute coefficients and feature names
importance = np.abs(ridge.coef_)
feature_names = X_train.columns

df_imp = pd.DataFrame({
    'feature': feature_names,
    'importance': importance
}).sort_values('importance', ascending=True)  # ascending for horizontal bars

# 3) Plot
plt.barh(df_imp['feature'], df_imp['importance'])
plt.xlabel('Absolute Coefficient Value')
plt.title('Feature Importances (Ridge Regression)')
plt.show()


In [ ]:
# 1) Actual vs Predicted
df1 = pd.DataFrame({
    'Actual CO2': y_test,
    'Predicted CO2': y_pred_ridge
})
fig1 = px.scatter(
    df1, x='Actual CO2', y='Predicted CO2',
    trendline='ols',
    trendline_color_override='black',
    labels={
      'Actual CO2':'Actual CO2 Emissions',
      'Predicted CO2':'Predicted CO2 Emissions'
    },
    title='Actual vs. Predicted CO2 Emissions'
)
fig1.show()


# 2) Residuals vs Fitted
residuals = y_test - y_pred_ridge
df2 = pd.DataFrame({
    'Fitted CO2': y_pred_ridge,
    'Residuals': residuals
})
fig2 = px.scatter(
    df2, x='Fitted CO2', y='Residuals',
    labels={
      'Fitted CO2':'Predicted (Fitted) CO2 of Ridge Regressions',
      'Residuals':'Residual'
    },
    title='Residuals vs. Fitted Values'
)
# add a horizontal zero‐error line
fig2.add_hline(y=0, line_dash='dash', line_color='black')
fig2.show()

#### Random Forest( Hypertuning parameters)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor(random_state=42)

param_dist = {
    'n_estimators': [100, 200, 500],
    'max_depth': [ None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'max_features': ['auto', 'sqrt', 0.5]
}

search = RandomizedSearchCV(
    rf,
    param_dist,
    n_iter=30,
    scoring='neg_root_mean_squared_error',
    cv=5,
    random_state=42,
    n_jobs=-1
)
search.fit(X_train_scaled, y_train)

print("Best params:", search.best_params_)
best_rf = search.best_estimator_

In [ ]:
### Random Forest

rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=20,
    n_jobs=-1,
    min_samples_split= 2,
    min_samples_leaf= 1,
    max_features='sqrt'
)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
# Metrics performance
rmse = root_mean_squared_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

# 2) Compute importances and their std across all trees
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

# 3) Sort features by importance
indices = np.argsort(importances)[::-1]
feature_names = X_train.columns




# 5) Plot with error bars
plt.figure(figsize=(8,6))
plt.title("Feature importances (with std)")
plt.bar(
    range(len(importances)),
    importances[indices],
    color="skyblue",
    yerr=std[indices],
    align="center"
)
plt.xticks(range(len(importances)), feature_names[indices], rotation=45, ha='right')
plt.ylabel("Mean importance")
plt.tight_layout()
plt.show()

#### XGB Regressor + Hypertuning parameters

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# XGBoost model
xgb = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,       # Lower learning rate for better generalization
    max_depth=5,
    reg_alpha=0.5,      # L1 regularization     
    reg_lambda=1.0,      # L2 regularzation    
    random_state=42,
    n_jobs=-1
)

# Train the model
xgb.fit(X_train_scaled, y_train)

# Predictions
y_pred_xgb = xgb.predict(X_test_scaled)

# Evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2 = r2_score(y_test, y_pred_xgb)
mae = mean_absolute_error(y_test, y_pred_xgb)

# Output
print(f"XGB RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

#### Gradient Boosting Regressor + Hypertuning parameters

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize Gradient Boosting Regressor with similar parameters
gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
)

# Fit model
gbr.fit(X_train_scaled, y_train)
y_pred_gbr = gbr.predict(X_test_scaled)

# Evaluate performance
rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))
r2_gbr = r2_score(y_test, y_pred_gbr)
mae_gbr = mean_absolute_error(y_test, y_pred_gbr)

# Compute feature importances
importances_gbr = gbr.feature_importances_
indices_gbr = np.argsort(importances_gbr)[::-1]
feature_names_gbr = X_train.columns
print(f" RMSE: {rmse_gbr:.3f}")
print(f"R²: {r2_gbr:.3f}")
print(f"MAE: {mae_gbr:.3f}")

#### K-Fold Cross-Validation for all models

In [ ]:
from sklearn.model_selection import cross_val_score
models = {'ridge': ridge, 'rf': rf, 'xgb': xgb, 'gbr': gbr}
cv_results = {}
for name, model in models.items():
    # 5-fold CV with R²
    scores = cross_val_score(model, X_train_scaled, y_train, scoring='r2', cv=5)
    cv_results[name] = scores
    print(f"{name} R_squared scores: {scores}")
    print(f"{name} mean R_squared:   {scores.mean():.4f}\n")

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve_px(model, name):
    train_sizes, train_scores, val_scores = learning_curve(
        model,
        X_train_scaled, y_train,
        cv=5,
        scoring='r2',
        train_sizes=np.linspace(0.1, 1.0, 5),
        n_jobs=-1
        )
    train_mean = train_scores.mean(axis=1)
    val_mean   = val_scores.mean(axis=1)

    df = pd.DataFrame({
        'training_set_size': np.concatenate([train_sizes, train_sizes]),
        'r2_score':          np.concatenate([train_mean, val_mean]),
        'split':             ['train'] * len(train_sizes) + ['validation'] * len(train_sizes)
    })

    fig = px.line(
        df, x='training_set_size',y='r2_score',color='split',
        markers=True,
        title=f'Learning Curve: {name}'
    )
    fig.update_layout(
        xaxis_title='Training Set Size',
        yaxis_title='R_squared Score',
        legend_title=''
    )
    fig.show()
plot_learning_curve_px(ridge, 'Ridge Regression')
plot_learning_curve_px(rf,    'Random Forest')

# ML: RQ: How significant is the relationship between latitude and temperature and can latitude be used to accurately predict temperatures?

## Visualizing Temperature Trends

In [ ]:
df_city['Year_bin'] = (df_city['Year'] // 10) * 10  
grouped = df_city.groupby(['Year_bin', 'Latitude_bin'], observed=True)['AverageTemperature'].mean().reset_index()
baseline = grouped.groupby('Latitude_bin', observed=True)['AverageTemperature'].mean().reset_index()
baseline.columns = ['Latitude_bin', 'BaselineTemp']
grouped = grouped.merge(baseline, on='Latitude_bin', how='left')
grouped['TempAnomaly'] = grouped['AverageTemperature'] - grouped['BaselineTemp']
anomaly_data = grouped.pivot(index='Latitude_bin', columns='Year_bin', values='TempAnomaly')
anomaly_data = anomaly_data.sort_index(ascending=True)

plt.figure(figsize=(14, 8))
sns.heatmap(anomaly_data, cmap="coolwarm", center=0, 
            cbar_kws={'label': 'Temperature Difference Compared to Average (°C)'}, 
            linewidths=0.2)
plt.title("Temperature Difference Heatmap by Latitude and Year")
plt.xlabel("Years")
plt.ylabel("Latitude")
plt.tight_layout()
plt.show()

## Correlation Matrix

In [ ]:
df_after_1950 = df_city[df_city['Year'] >= 1950].copy()
data = df_after_1950[['AverageTemperature', 'Latitude', 'Longitude']].copy()
correlation_matrix = data.corr(method='pearson')  

plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Feature Correlation with Temperature")
plt.tight_layout()
plt.show()

## Regression Models - Latitude Only

In [ ]:
features_latitude = df_after_1950[['Latitude']]
features_latitude_longitude = df_after_1950[['Latitude', 'Longitude']]
temperature = df_after_1950['AverageTemperature']

X_train_latitude, X_test_latitude, y_train, y_test = train_test_split(features_latitude, temperature, test_size=0.2, random_state=42)
X_train_both_features, X_test_both_features, _, _ = train_test_split(features_latitude_longitude, temperature, test_size=0.2, random_state=42)

models = {
    "Linear Regression Model": LinearRegression(),
    "Random Forest Model": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "Gradient Boosting Model": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost Model": XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)
}

results = []
print("Model Performance (Latitude Only):")
for name, model in models.items():
    model.fit(X_train_latitude, y_train)
    preds = model.predict(X_test_latitude)
    rmse = root_mean_squared_error(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    results.append({'Model': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
    print(f"{name}: RMSE = {rmse:.2f} °C | MAE = {mae:.2f} °C | R² = {r2:.2f}")

best_model_result = min(results, key=lambda x: x['RMSE'])
best_model_name = best_model_result['Model']
best_model = models[best_model_name]
print(f"\nBest Model (Latitude): {best_model_name}")

## Regression Models - Latitude + Longitude

In [ ]:
best_model.fit(X_train_both_features, y_train)
predictions_latitude_longitude = best_model.predict(X_test_both_features)
rmse_latitude_longitude = root_mean_squared_error(y_test, predictions_latitude_longitude)
r2_latitude_longitude = r2_score(y_test, predictions_latitude_longitude)
mae_latitude_longitude = mean_absolute_error(y_test, predictions_latitude_longitude)

print(f"\n{best_model_name} Performance (Latitude + Longitude):")
print(f"RMSE = {rmse_latitude_longitude:.2f} °C | R² = {r2_latitude_longitude:.2f} | MAE = {mae_latitude_longitude:.2f} °C")

## Scatter Plot and Hexbin - Actuals vs Predicted

In [ ]:
# Hexbin
plt.figure(figsize=(8, 6))
plt.hexbin(y_test, predictions_latitude_longitude, gridsize=50, cmap='viridis', bins='log')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')
plt.xlabel("Actual Temperatures (°C)")
plt.ylabel("Predicted Temperatures (°C)")
plt.title(f"{best_model_name} Predictions vs Actuals")
plt.colorbar(label='log(N)')
plt.tight_layout()
plt.show()

# Scatterplot
df = pd.DataFrame({
    'Actual Temperatures (°C)': y_test,
    'Predicted Temperatures (°C)': predictions_latitude_longitude
})

fig = px.scatter(
    df,
    x='Actual Temperatures (°C)',
    y='Predicted Temperatures (°C)',
    trendline_color_override='red',
    labels={
        'Actual Temperatures (°C)': 'Actual Temperatures (°C)',
        'Predicted Temperatures (°C)': 'Predicted Temperatures (°C)'
    },
    title=f'{best_model_name} Predictions vs Actuals'
)
fig.show()


In [ ]:
df = pd.read_csv('data/owid-co2-data.csv')

In [ ]:
df['gdp_per_capita'] = df['gdp'] / df['population']

columns_of_interest = [
    'co2', 'co2_per_capita', 'gdp', 'gdp_per_capita', 
    'energy_per_capita', 'coal_co2', 'oil_co2', 'gas_co2',
    'cement_co2', 'land_use_change_co2', 'consumption_co2'
]

continental_entities = [
    'Africa', 
    'Asia', 
    'Europe', 
    'European Union (28)',
    'North America',
    'South America',
    'Oceania'
]
df_copy = df.copy(deep=True)
df_continental = df_copy[df_copy['country'].isin(continental_entities)]
# remove the gdp and gdp_per_capita columns from the df_continental
df_continental = df_continental[columns_of_interest].drop(columns=['gdp', 'gdp_per_capita'])

df = df.dropna(subset=columns_of_interest)



income_group_entities = [
    'High-income countries', 'Low-income countries',
    'Lower-middle-income countries', 'Upper-middle-income countries',
    'Least developed countries (Jones et al.)'
]

df_income_group = df[df['country'].isin(income_group_entities)].copy()

countries = [
    'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
    'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria',
    'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
    'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
    'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei',
    'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
    'Canada', 'Cape Verde', 'Central African Republic', 'Chad', 'Chile',
    'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica',
    "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
    'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic',
    'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
    'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland',
    'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
    'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea',
    'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hungary',
    'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq',
    'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan',
    'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 'Kuwait',
    'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho',
    'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg',
    'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali',
    'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius', 'Mexico',
    'Micronesia (country)', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro',
    'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru',
    'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua', 'Niger',
    'Nigeria', 'North Korea', 'North Macedonia', 'Norway', 'Oman',
    'Pakistan', 'Palau', 'Panama', 'Papua New Guinea', 'Paraguay',
    'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar',
    'Romania', 'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia',
    'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia',
    'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore',
    'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa',
    'South Korea', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan',
    'Suriname', 'Sweden', 'Switzerland', 'Syria', 'Tajikistan',
    'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga',
    'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Tuvalu',
    'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States',
    'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela', 'Vietnam',
    'Yemen', 'Zambia', 'Zimbabwe'
]

df_countries = df[df['country'].isin(countries)].copy()

def safe_qcut(group):
    try:
        return pd.qcut(group, q=3, labels=['low', 'middle', 'high'], duplicates='drop')
    except ValueError:
        # For countries with insufficient unique values, assign a default label
        return pd.Series(['low'] * len(group), index=group.index)

df_countries['income_bin'] = df_countries.groupby('country')['gdp_per_capita'].transform(safe_qcut)


df_world = df_countries.groupby(['year']).agg({
    'co2': 'sum',
    'co2_per_capita': 'mean',
    'gdp': 'sum',
    'gdp_per_capita': 'mean',
    'energy_per_capita': 'mean',
    'coal_co2': 'sum',
    'oil_co2': 'sum',
    'gas_co2': 'sum',
    'cement_co2': 'sum',
    'land_use_change_co2': 'sum',
    'consumption_co2': 'sum'
}).reset_index()


In [ ]:
def calculate_yearly_terciles(df, income_col='gdp_per_capita', 
                            year_col='year', country_col='country'):
    df = df.copy()
    df['income_bin'] = np.nan
    df = df[(df[income_col].notna()) & (df[income_col] > 0)]

    for year in df[year_col].unique():
        year_mask = df[year_col] == year
        year_data = df[year_mask].copy()
        
        # Sort by GDP values (ascending)
        year_data = year_data.sort_values(income_col)
        
        # Calculate GDP thresholds that divide into equal groups
        n = len(year_data)
        low_threshold = year_data[income_col].iloc[n//3 - 1]  # -1 because Python is 0-indexed
        high_threshold = year_data[income_col].iloc[2*n//3 - 1]
        
        # Assign bins based on GDP values
        df.loc[year_mask, 'income_bin'] = np.where(
            df.loc[year_mask, income_col] <= low_threshold, 'low',
            np.where(
                df.loc[year_mask, income_col] <= high_threshold, 'middle', 'high'
            )
        )
        
        # Verification for debugging
        if year == df[year_col].min():
            print(f"\n=== Year {year} Bin Assignment ===")
            print(f"Low threshold GDP: {low_threshold:.2f}")
            print(f"High threshold GDP: {high_threshold:.2f}")
            print("\nSample assignments:")
            sample = df[year_mask].sort_values(income_col)
            print(sample[[country_col, income_col, 'income_bin']].head(3))
            print(sample[[country_col, income_col, 'income_bin']].tail(3))
    
    # Ensure correct categorical ordering
    df['income_bin'] = pd.Categorical(
        df['income_bin'], 
        categories=['low', 'middle', 'high'], 
        ordered=True
    )
    
    return df

def analyze_income_mobility(df, country_col='country', year_col='year'):

    # Ensure proper sorting
    df = df.sort_values([country_col, year_col])
    
    # 1. Calculate transition statistics
    transitions = df.groupby(country_col)['income_bin'].agg([
        ('initial_bin', 'first'),
        ('final_bin', 'last'),
        ('unique_bins', 'nunique'),
        ('max_bin', 'max'),
        ('min_bin', 'min')
    ])
    
    # Calculate mobility direction
    transitions['mobility'] = np.where(
        transitions['initial_bin'] < transitions['final_bin'], 'up',
        np.where(transitions['initial_bin'] > transitions['final_bin'], 'down', 'stable'))
    
    # 2. Verify tercile distribution
    print("=== Yearly Bin Distribution Verification ===")
    yearly_counts = pd.crosstab(df[year_col], df['income_bin'])
    print(yearly_counts)
    
    # Check if counts are balanced
    if not all(yearly_counts.nunique(axis=1) == 1):
        print("\nWarning: Some years have unequal bin counts due to total country count not being divisible by 3")
    
    # 3. Visualization
    plt.figure(figsize=(14, 8))
    
    # Plot 1: Distribution of income bins over time
    plt.subplot(2, 2, 1)
    sns.countplot(data=df, x=year_col, hue='income_bin', palette='viridis')
    plt.title('Income Bin Distribution by Year')
    plt.xticks(rotation=45)
    
    # Plot 2: Mobility summary
    plt.subplot(2, 2, 2)
    transitions['mobility'].value_counts().plot(kind='bar', color=['green', 'gray', 'red'])
    plt.title('Overall Mobility Direction')
    plt.xlabel('Mobility Type')
    
    # Plot 3: Example trajectories
    plt.subplot(2, 1, 2)
    sample_countries = ['Poland', 'United States', 'China', 'India', 'Brazil']
    for country in sample_countries:
        country_data = df[df[country_col] == country]
        plt.plot(country_data[year_col].astype(str), 
                country_data['income_bin'].cat.codes, 
                marker='o', label=country)
    
    plt.yticks(ticks=[0, 1, 2], labels=['low', 'middle', 'high'])
    plt.title('Country Income Trajectories')
    plt.xlabel('Year')
    plt.ylabel('Income Bin')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    return transitions

# Calculate yearly income bins
df_with_bins = calculate_yearly_terciles(df_countries)


In [ ]:
def plot_correlation_heatmaps(df, group_col=None):
    # Define feature sets
    general_features = ['co2', 'gdp', 'coal_co2', 'oil_co2', 
                      'gas_co2', 'cement_co2', 'energy']
    
    per_capita_features = ['co2_per_capita', 'gdp_per_capita', 
                         'energy_per_capita', 'coal_co2_per_capita',
                         'oil_co2_per_capita', 'gas_co2_per_capita',
                         'cement_co2_per_capita']
    
    # Filter to only include features that exist in the dataframe
    general_features = [f for f in general_features if f in df.columns]
    per_capita_features = [f for f in per_capita_features if f in df.columns]
    
    if not general_features and not per_capita_features:
        print("No valid features found for correlation analysis")
        return
    
    if group_col is None:
        # Create figure for global correlations
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
        
        # Plot general correlations
        if general_features:
            corr_general = df[general_features].corr()
            sns.heatmap(corr_general, annot=True, cmap='coolwarm', center=0,
                      fmt='.2f', annot_kws={'size': 9}, ax=ax1)
            ax1.set_title('General Metrics Correlations', fontsize=20)
            ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
        
        # Plot per capita correlations
        if per_capita_features:
            corr_per_capita = df[per_capita_features].corr()
            sns.heatmap(corr_per_capita, annot=True, cmap='coolwarm', center=0,
                       fmt='.2f', annot_kws={'size': 9}, ax=ax2)
            ax2.set_title('Per Capita Metrics Correlations', fontsize=20)
            ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')

        plt.suptitle('Global Correlation Matrices', fontsize=20)
        plt.tight_layout()
        plt.show()
        
    else:
        # Grouped correlations
        if group_col not in df.columns:
            print(f"Grouping column '{group_col}' not found in DataFrame")
            return
            
        for name, group in df.groupby(group_col):
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
            
            # Plot general correlations
            if general_features:
                corr_general = group[general_features].corr()
                sns.heatmap(corr_general, annot=True, cmap='coolwarm', center=0,
                           fmt='.2f', annot_kws={'size': 9}, ax=ax1)
                ax1.set_title(f'General Metrics for {name}', fontsize=20)
                ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
            
            # Plot per capita correlations
            if per_capita_features:
                corr_per_capita = group[per_capita_features].corr()
                sns.heatmap(corr_per_capita, annot=True, cmap='coolwarm', center=0,
                            fmt='.2f', annot_kws={'size': 14}, ax=ax2)
                ax2.set_title(f'Per Capita Metrics for {name}', fontsize=20)
                ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')

            plt.suptitle(f'Correlation Matrices for {group_col}: {name}', fontsize=20)
            plt.tight_layout()
            plt.show()

def plot_per_capita_correlations_by_income(df, income_col='income_bin'):
    # Define our specific per capita metrics of interest
    per_capita_metrics = [
        'co2_per_capita',
        'gdp_per_capita', 
        'energy_per_capita'
    ]
    
    # Filter to only include metrics that exist in the dataframe
    per_capita_metrics = [m for m in per_capita_metrics if m in df.columns]
    
    if not per_capita_metrics:
        print("No per capita metrics found in DataFrame")
        return
    
    if income_col not in df.columns:
        print(f"Income group column '{income_col}' not found in DataFrame")
        return
    
    # Get unique income groups
    income_groups = df[income_col].unique()
    
    # Create a figure with subplots for each income group
    fig, axes = plt.subplots(1, len(income_groups), figsize=(6*len(income_groups), 5))
    
    # If only one income group, axes won't be an array
    if len(income_groups) == 1:
        axes = [axes]
    
    for ax, income_group in zip(axes, income_groups):
        # Filter data for this income group
        group_data = df[df[income_col] == income_group]
        
        # Calculate correlations
        corr_matrix = group_data[per_capita_metrics].corr()
        
        # Plot heatmap
        sns.heatmap(
            corr_matrix,
            annot=True,
            cmap='coolwarm',
            center=0,
            vmin=-1,
            vmax=1,
            fmt='.2f',
            annot_kws={'size': 15},
            ax=ax
        )
        
        ax.set_title(
            f'{income_group}\n(n={len(group_data)})',
            fontsize=20,
            pad=20
        )
        
        # Rotate x-axis labels
        ax.set_xticklabels(
            ax.get_xticklabels(),
            rotation=45,
            ha='right',
            fontsize=12
        )
        
        # Rotate y-axis labels
        ax.set_yticklabels(
            ax.get_yticklabels(),
            rotation=0,
            fontsize=12
        )
    
    plt.suptitle(
        'Per Capita Metrics Correlation by Income Group',
        fontsize=22,
        y=1.05
    )
    plt.tight_layout()
    plt.show()

In [ ]:
plot_correlation_heatmaps(df_world)
plot_per_capita_correlations_by_income(df_countries)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


def run_regression_models(df, group_col, features=None, target=None):
    if features is None:
        features = ['gdp_per_capita', 'energy_per_capita', 
                   'coal_co2', 'oil_co2', 'gas_co2', 'cement_co2']
    if target is None:
        target = 'co2_per_capita'

    df = df[df['country'] != 'Least developed countries (Jones et al.)']
    
    groups = df[group_col].unique()
    results = []
    model_details = []
    
    for group in groups:
        group_df = df[df[group_col] == group].dropna(subset=features + [target])
        if len(group_df) < 20:  # Skip groups with insufficient data
            continue
            
        X = group_df[features]
        y = group_df[target]
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        
        models = {
            'Linear Regression': LinearRegression(),
            'Random Forest': RandomForestRegressor(random_state=42),
            'XGBoost': XGBRegressor(random_state=42)
        }
        
        for name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
            # Store performance results
            results.append({
                'Group': group,
                'Model': name,
                'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
                'R2': r2_score(y_test, y_pred),
                'Samples': len(group_df)
            })
            
            # Store model coefficients/importance
            if name == 'Linear Regression':
                coef_data = {
                    'Group': group,
                    'Model': name,
                    'Type': 'Coefficient',
                    'Features': features,
                    'Values': model.coef_,
                    'Intercept': model.intercept_
                }
            else:  # Tree-based models
                coef_data = {
                    'Group': group,
                    'Model': name,
                    'Type': 'Feature Importance',
                    'Features': features,
                    'Values': model.feature_importances_,
                    'Intercept': None
                }
            model_details.append(coef_data)

    # Convert results to DataFrames
    results_df = pd.DataFrame(results)
    details_df = pd.DataFrame(model_details)
    
    # Print performance results
    print("\n=== Model Performance Results ===")
    print(results_df.to_string(index=False))
    
    # Print detailed coefficients/importance
    print("\n=== Model Coefficients/Feature Importance ===")
    for group in details_df['Group'].unique():
        print(f"\n--- {group} ---")
        group_details = details_df[details_df['Group'] == group]
        
        for _, row in group_details.iterrows():
            print(f"\n{row['Model']} ({row['Type']}):")
            for feature, value in zip(row['Features'], row['Values']):
                print(f"{feature}: {value:.4f}")
            if row['Intercept'] is not None:
                print(f"Intercept: {row['Intercept']:.4f}")

    # Plot results
    plt.figure(figsize=(12, 6))
    sns.barplot(data=results_df, x='Group', y='R2', hue='Model')
    plt.title(f'Model Performance (R²) by {group_col.replace("_", " ").title()}')
    plt.ylabel('R² Score')
    plt.xlabel(group_col.replace("_", " ").title())
    plt.xticks(rotation=45, ha='right')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return results_df, details_df


In [ ]:
income_bin_results = run_regression_models(df_countries, 'income_bin', ['gdp'], 'co2')

In [ ]:
def plot_sector_trends(df, group_col=None):
    sectors = ['coal_co2', 'oil_co2', 'gas_co2', 'cement_co2']
    sector_names = [s.replace('_co2', '').replace('_', ' ').title() for s in sectors]
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    df = df[df['year'] >= 1900].copy()  # Filter for years >= 1900
    
    # Calculate percentage contribution
    df['total_co2'] = df[sectors].sum(axis=1)
    for sector in sectors:
        df[f'{sector}_pct'] = df[sector] / df['total_co2'] * 100
    
    if group_col is None:
        # Global trends
        plt.figure(figsize=(12, 6))
        
        # Calculate mean percentages by year
        trend_data = df.groupby('year')[[f'{s}_pct' for s in sectors]].mean()
        
        # Stacked area plot
        plt.stackplot(trend_data.index, trend_data.values.T,
                     labels=sector_names, colors=colors, alpha=0.8)
        
        plt.title('Global CO₂ Emissions Trends by Sector')
        plt.ylabel('Percentage Contribution')
        plt.xlabel('Year')
        plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
        plt.grid(True, alpha=0.3)
        plt.xlim(trend_data.index.min(), trend_data.index.max())
        plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust right margin for legend
        plt.show()
        
    else:
        # Grouped trends
        groups = df[group_col].unique()
        n_groups = len(groups)
        n_cols = min(3, n_groups)
        n_rows = (n_groups + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 5*n_rows))
        axes = axes.flatten() if n_groups > 1 else [axes]
        
        for i, group in enumerate(groups):
            ax = axes[i]
            group_data = df[df[group_col] == group]
            trend_data = group_data.groupby('year')[[f'{s}_pct' for s in sectors]].mean()
            
            ax.stackplot(trend_data.index, trend_data.values.T,
                        labels=sector_names, colors=colors, alpha=0.8)
            
            ax.set_title(f'CO2 Emissions Trends - {group}')
            ax.set_ylabel('Percentage Contribution')
            ax.set_xlabel('Year')
            ax.grid(True, alpha=0.3)
            ax.set_xlim(trend_data.index.min(), trend_data.index.max())
        
        # Add single legend for all subplots
        handles, labels = axes[0].get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper right', 
                  bbox_to_anchor=(0.98, 0.98), title='Sectors')
        
        # Hide empty subplots if any
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
            
        plt.tight_layout(rect=[0, 0, 0.95, 1])  # Adjust right margin
        plt.show()

In [ ]:
def run_regression_analysis(df, target_col, feature_cols, group_col=None, test_size=0.2, val_size=0.25, random_state=42):
    # Initialize models
    models = {
        'Linear Regression': LinearRegression()
    }
    
    # Remove rows with NaN in target or features
    df = df.dropna(subset=[target_col] + feature_cols)

    # Preprocessing
    X = df[feature_cols]
    y = df[target_col]
    
    # Standardize features (except for tree-based models which don't need it)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Full train-val-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, random_state=random_state)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=random_state)
    print(f"Train samples: {len(y_train)}, Val samples: {len(y_val)}, Test samples: {len(y_test)}")
    results = []
    coefficients = []
    
    if group_col is None:
        # Single dataset analysis
        results = _evaluate_models(models, X_train, X_val, X_test, y_train, y_val, y_test)
        _plot_results(results, X_test, y_test, models)
    else:
        # Grouped analysis - create faceted plots
        groups = df[group_col].unique()
        n_groups = len(groups)
        n_cols = min(3, n_groups)
        n_rows = (n_groups + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
        axes = axes.flatten() if n_groups > 1 else [axes]
        
        for i, group in enumerate(groups):
            group_df = df[df[group_col] == group]
            X_group = group_df[feature_cols]
            y_group = group_df[target_col]
            
            # Train-test split for group
            X_train, X_test, y_train, y_test = train_test_split(
                scaler.transform(X_group), y_group, test_size=test_size, random_state=random_state)
            X_train, X_val, y_train, y_val = train_test_split(
                X_train, y_train, test_size=val_size, random_state=random_state)
            
            # Evaluate models for this group
            group_results = _evaluate_models(models, X_train, X_val, X_test, y_train, y_val, y_test)
            results.extend(group_results)
            
            # Plot for this group
            ax = axes[i]
            _plot_group_results(ax, group_results, group)
        
        # Hide empty subplots
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Print comprehensive metrics
    _print_metrics_report(pd.DataFrame(results))
    
    return pd.DataFrame(results)

def _evaluate_models(models, X_train, X_val, X_test, y_train, y_val, y_test):
    results = []
    
    for name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        
        # Predictions
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
        
        # Calculate metrics
        val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        val_mae = mean_absolute_error(y_val, y_val_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        val_r2 = r2_score(y_val, y_val_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        results.append({
            'Model': name,
            'Val_RMSE': val_rmse,
            'Test_RMSE': test_rmse,
            'Val_MAE': val_mae,
            'Test_MAE': test_mae,
            'Val_R2': val_r2,
            'Test_R2': test_r2
        })
    
    return results, model.details

def _plot_results(results, X_test, y_test, models):
    # Metrics comparison
    metrics_df = pd.DataFrame(results)
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Model', y='Test_R2', data=metrics_df)
    plt.title('Model Comparison by R² Score')
    plt.ylabel('R² Score (Test Set)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Actual vs Predicted for best model
    best_model_name = metrics_df.loc[metrics_df['Test_R2'].idxmax(), 'Model']
    best_model = models[best_model_name]
    y_pred = best_model.predict(X_test)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted - {best_model_name}')
    plt.tight_layout()
    plt.show()

def _plot_group_results(ax, group_results, group_name):
    metrics_df = pd.DataFrame(group_results)
    sns.barplot(x='Model', y='Test_R2', data=metrics_df, ax=ax)
    ax.set_title(f'{group_name} - Model Comparison')
    ax.set_ylabel('R² Score (Test Set)')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

def _print_metrics_report(results_df):
    print("\n=== Regression Analysis Report ===")
    print("\nModel Performance Metrics:")
    print(results_df.groupby('Model')[['Test_RMSE', 'Test_MAE', 'Test_R2']].mean())
    
    print("\nBest Model by Metric:")
    print(f"Best R²: {results_df.loc[results_df['Test_R2'].idxmax(), 'Model']}")
    print(f"Best RMSE: {results_df.loc[results_df['Test_RMSE'].idxmin(), 'Model']}")
    print(f"Best MAE: {results_df.loc[results_df['Test_MAE'].idxmin(), 'Model']}")

In [ ]:
def run_regression_analysis(df, target_col, feature_cols, group_col=None, test_size=0.2, val_size=0.01, random_state=42):
    # Initialize models
    models = {
        'Linear Regression': LinearRegression(),
    }
    
    # Remove rows with NaN in target or features
    df = df.dropna(subset=[target_col] + feature_cols).copy()
    
    # Initialize storage
    all_results = []
    lr_coefficients = []
    group_metrics = {}  # To store metrics per group
    
    # Preprocessing
    X = df[feature_cols]
    y = df[target_col]
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    if group_col is None:
        # Single dataset analysis
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=test_size, random_state=random_state)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=val_size, random_state=random_state)
        
        results, coef_data = _evaluate_models(models, X_train, X_val, X_test, y_train, y_val, y_test, feature_cols)
        all_results.extend(results)
        
        if coef_data:
            lr_coefficients.append(coef_data)
        
        _plot_results(results, X_test, y_test, models)
    else:
        # Grouped analysis
        groups = df[group_col].unique()
        n_groups = len(groups)
        n_cols = min(3, n_groups)
        n_rows = (n_groups + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
        axes = axes.flatten() if n_groups > 1 else [axes]
        
        for i, group in enumerate(groups):
            group_df = df[df[group_col] == group]
            if len(group_df) < 20:  # Skip small groups
                print(f"\nSkipping group {group} - insufficient data (n={len(group_df)})")
                continue
                
            X_group = group_df[feature_cols]
            y_group = group_df[target_col]
            
            # Train-test split for group
            X_train, X_test, y_train, y_test = train_test_split(
                scaler.transform(X_group), y_group, test_size=test_size, random_state=random_state)
            X_train, X_val, y_train, y_val = train_test_split(
                X_train, y_train, test_size=val_size, random_state=random_state)
            
            # Evaluate models
            group_results, coef_data = _evaluate_models(
                models, X_train, X_val, X_test, y_train, y_val, y_test, feature_cols)
            
            if group_results:
                all_results.extend(group_results)
                group_metrics[group] = group_results  # Store metrics by group
                
                # Store coefficients
                if coef_data:
                    coef_data['Group'] = group
                    lr_coefficients.append(coef_data)
                
                # Plot for this group
                ax = axes[i]
                _plot_group_results(ax, group_results, group)
        
        # Hide empty subplots
        for j in range(i+1, len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Print comprehensive metrics report
    print("\n=== REGRESSION ANALYSIS REPORT ===")
    
    # Print performance metrics per group
    if group_col:
        print("\n=== PERFORMANCE METRICS BY GROUP ===")
        for group, metrics in group_metrics.items():
            print(f"\n--- {group} ---")
            metrics_df = pd.DataFrame(metrics)
            print(metrics_df[['Model', 'Test_RMSE', 'Test_MAE', 'Test_R2']].to_string(index=False, float_format="%.3f"))
    
    # Print overall metrics
    print("\n=== OVERALL PERFORMANCE ===")
    results_df = pd.DataFrame(all_results)
    _print_metrics_report(results_df)
    
    # Print Linear Regression coefficients
    if lr_coefficients:
        lr_coef_df = pd.DataFrame(lr_coefficients)
        print("\n=== LINEAR REGRESSION COEFFICIENTS ===")
        
        # Create a clean display of all coefficients
        if group_col:
            # For grouped analysis
            coef_columns = ['Group'] + feature_cols + ['Intercept']
            print(lr_coef_df[coef_columns].to_string(float_format="%.4f", index=False))
        else:
            # For single analysis
            coef_columns = feature_cols + ['Intercept']
            print(lr_coef_df[coef_columns].to_string(float_format="%.4f", index=False))
    
    return results_df, lr_coef_df if lr_coefficients else None

def _evaluate_models(models, X_train, X_val, X_test, y_train, y_val, y_test, feature_names):

    results = []
    coef_data = None
    
    for name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        
        # Predictions
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
        
        # Calculate metrics
        metrics = {
            'Model': name,
            'Val_RMSE': np.sqrt(mean_squared_error(y_val, y_val_pred)),
            'Test_RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
            'Val_MAE': mean_absolute_error(y_val, y_val_pred),
            'Test_MAE': mean_absolute_error(y_test, y_test_pred),
            'Val_R2': r2_score(y_val, y_val_pred),
            'Test_R2': r2_score(y_test, y_test_pred)
        }
        results.append(metrics)
        
        # Store coefficients for Linear Regression
        if name == 'Linear Regression':
            coef_data = {}
            # Store all coefficients
            for feature, coef in zip(feature_names, model.coef_):
                coef_data[feature] = coef
            coef_data['Intercept'] = model.intercept_
    
    return results, coef_data

def _print_metrics_report(results_df):
    print("\nModel Performance Metrics:")
    print(results_df.groupby('Model')[['Test_RMSE', 'Test_MAE', 'Test_R2']].mean().to_string(float_format="%.3f"))
    
    print("\nBest Model by Metric:")
    print(f"Best R²: {results_df.loc[results_df['Test_R2'].idxmax(), 'Model']} ({results_df['Test_R2'].max():.3f})")
    print(f"Best RMSE: {results_df.loc[results_df['Test_RMSE'].idxmin(), 'Model']} ({results_df['Test_RMSE'].min():.3f})")
    print(f"Best MAE: {results_df.loc[results_df['Test_MAE'].idxmin(), 'Model']} ({results_df['Test_MAE'].min():.3f})")
    

def _plot_results(results, X_test, y_test, models):
    # Metrics comparison
    metrics_df = pd.DataFrame(results)

    # Actual vs Predicted for best model
    best_model_name = metrics_df.loc[metrics_df['Test_R2'].idxmax(), 'Model']
    best_model = models[best_model_name]
    y_pred = best_model.predict(X_test)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted - {best_model_name}')
    plt.tight_layout()
    plt.xscale('log')
    plt.yscale('log')
    plt.show()

# def _plot_group_results(ax, group_results, group_name):
#     """Plot results for a single group"""
#     metrics_df = pd.DataFrame(group_results)
#     sns.barplot(x='Model', y='Test_R2', data=metrics_df, ax=ax)
#     ax.set_title(f'{group_name} - Model Comparison')
#     ax.set_ylabel('R² Score (Test Set)')
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

def _print_metrics_report(results_df):
    print("\n=== Regression Analysis Report ===")
    print("\nModel Performance Metrics:")
    print(results_df.groupby('Model')[['Test_RMSE', 'Test_MAE', 'Test_R2']].mean())
    
    print("\nBest Model by Metric:")
    print(f"Best R²: {results_df.loc[results_df['Test_R2'].idxmax(), 'Model']}")
    print(f"Best RMSE: {results_df.loc[results_df['Test_RMSE'].idxmin(), 'Model']}")
    print(f"Best MAE: {results_df.loc[results_df['Test_MAE'].idxmin(), 'Model']}")

In [ ]:
df_world_per_capita = df_countries[['co2_per_capita', 'energy_per_capita', 'gdp_per_capita','year']].groupby('year').mean().reset_index()
df_world_total = df_countries.groupby('year').sum().reset_index()


# For grouped analysis (returns coefficients)
metrics_df, coef_df = run_regression_analysis(
    df_countries,
    target_col='co2_per_capita',
    feature_cols=['gdp_per_capita'],
    random_state=20,
    test_size=0.5
)

# For overall analysis (no coefficients)
metrics_df, _ = run_regression_analysis(
    df_countries,
    target_col='co2_per_capita',
    feature_cols=['gdp_per_capita'],
    group_col='income_bin',
    test_size=0.5,
    random_state=20
)

In [ ]:
# %%
import pandas as pd


df_major = pd.read_csv('GlobalLandTemperaturesByMajorCity.csv', parse_dates=['dt'])
df_major = df_major[df_major['dt'].dt.year >= 1950]  


print(df_major.isnull().sum())

# %% [markdown]
# # Getting the best cities with each cluster through the use of k-means 

# %%
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans



cities = df_major.groupby('City').agg({
    'Country': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    'AverageTemperature': 'count'  
}).reset_index()
cities.rename(columns={'AverageTemperature': 'DataPoints'}, inplace=True)

#ONLY CITIES WITH <80 completeness
min_data_points = len(df_major['dt'].unique()) * 0.8
qualified_cities = cities[cities['DataPoints'] >= min_data_points].copy()


#converter coord to floats
def coord_to_float(coord):
    if isinstance(coord, str):
        try:
            value = float(coord[:-1])
            direction = coord[-1].upper()
            if direction in ['W', 'S']:
                return -value
            return value
        except:
            return np.nan
    return coord


qualified_cities['Latitude'] = qualified_cities['Latitude'].apply(coord_to_float)
qualified_cities['Longitude'] = qualified_cities['Longitude'].apply(coord_to_float)

#DROP BS LAT OR LONG 
qualified_cities = qualified_cities.dropna(subset=['Latitude', 'Longitude'])



def lat_to_float(x):
    if isinstance(x, str):
        return float(x[:-1]) * (1 if x[-1] == 'N' else -1)
    return x
qualified_cities['Latitude'] = qualified_cities['Latitude'].apply(lat_to_float)


kmeans = KMeans(n_clusters=30, random_state=42)  

qualified_cities['Longitude'] = qualified_cities['Longitude'].apply(coord_to_float)


print(qualified_cities[['City', 'Latitude', 'Longitude']].head())

#KMEANS
kmeans = KMeans(n_clusters=30, random_state=42)
qualified_cities['Cluster'] = kmeans.fit_predict(qualified_cities[['Latitude', 'Longitude']])



selected_cities = qualified_cities.loc[
    qualified_cities.groupby('Cluster')['DataPoints'].idxmax()
]


print(f"Selected {len(selected_cities)} cities:")
print(selected_cities[['City', 'Country', 'Latitude', 'Longitude']].sort_values('Latitude'))


df_selected = df_major[df_major['City'].isin(selected_cities['City'])]
print(f"\nFiltered dataset shape: {df_selected.shape}")

selected_cities.to_csv('selected_cities.csv', index=False)
df_selected.to_csv('filtered_temperatures.csv', index=False)

df_major = df_major[df_major['City'].isin(selected_cities['City'])]

# %% [markdown]
# # Missing data with the use of KNN respects local similarities

# %%
from sklearn.impute import KNNImputer

#MISSING KNN DATA INFIL 

pivot = df_major.pivot_table(index=['dt', 'City'], 
                      values=['AverageTemperature', 'AverageTemperatureUncertainty'])


imputer = KNNImputer(n_neighbors=5)
imputed_values = imputer.fit_transform(pivot)


df_imputed = (
    pd.DataFrame(imputed_values, 
                columns=pivot.columns, 
                index=pivot.index)
    .reset_index()
)

#Merge
other_cols = df_major.columns.difference(['AverageTemperature', 'AverageTemperatureUncertainty'])
df_imputed = df_imputed.merge(
    df_major[other_cols].drop_duplicates(['dt', 'City']),
    on=['dt', 'City'],
    how='left'
)


print(df_imputed.head())

# %%
print(df_imputed.isnull().sum())

# %%
print(df_imputed.head())

# %%
df_imputed = df_imputed.round({
    'AverageTemperature': 1,       
    'AverageTemperatureUncertainty': 1,
})


# %% [markdown]
# # QUICK DATA VISIUALIZATION 

# %%
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


df_major = pd.read_csv('GlobalLandTemperaturesByCity.csv')


unique_cities = df_major['City'].nunique()
print(f"Total unique cities in dataset: {unique_cities:,}")


cities_per_country = df_major.groupby('Country')['City'].nunique().sort_values(ascending=False)
print("\nCities per country (Top 20):")
print(cities_per_country.head(20))


def convert_coord(coord):
    if isinstance(coord, str):
        value = float(coord[:-1])
        return -value if coord[-1] in ['W', 'S'] else value
    return coord

coord_df = df_major.groupby('City').first()[['Latitude', 'Longitude']].applymap(convert_coord)


plt.figure(figsize=(15, 8))


ax = plt.subplot(111, projection='3d')
ax.scatter(
    coord_df['Longitude'],
    coord_df['Latitude'],
    c=coord_df.index.map(hash) % 100,  
    s=5,
    alpha=0.6
)

ax.set(
    xlabel='Longitude',
    ylabel='Latitude',
    title=f'Global Distribution of {unique_cities:,} Weather Stations'
)
plt.tight_layout()
plt.savefig('city_distribution.png', dpi=300)
plt.show()


full_city_list = df_major[['City', 'Country', 'Latitude', 'Longitude']].drop_duplicates()
full_city_list.to_csv('all_cities_list.csv', index=False)

print("\nAdditional Stats:")
print(f"- Cities in Northern Hemisphere: {(coord_df['Latitude'] > 0).sum():,}")
print(f"- Cities in Southern Hemisphere: {(coord_df['Latitude'] < 0).sum():,}")
print(f"- Coastal cities (estimated): {(abs(coord_df['Longitude']) < 5).sum():,}")

# %%


# %%


# %%
uuunique_cities = df_imputed['City'].nunique()
print(f"Total unique cities in dataset: {uuunique_cities:,}")

# %%
print(df_major['City'].nunique())

# %%


# %%
df_selected = df_imputed[df_imputed['City'].isin(selected_cities['City'])]

# %%


unique_cities = df_imputed['City'].nunique()
print(f"Total unique cities in dataset: {unique_cities:,}")

# %%
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import numpy as np


selected_cities = pd.read_csv('selected_cities.csv')  


plt.figure(figsize=(16, 10))
ax = plt.axes(projection=ccrs.Robinson())
ax.set_global()
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.OCEAN, facecolor='azure')
ax.add_feature(cfeature.COASTLINE, edgecolor='dimgray')
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray')


colors = plt.cm.plasma((selected_cities['Latitude'] + 90) / 180)  
scatter = ax.scatter(
    selected_cities['Longitude'],
    selected_cities['Latitude'],
    c=colors,
    s=120,
    transform=ccrs.PlateCarree(),
    edgecolors='black',
    linewidths=0.5,
    alpha=0.9
)


label_cities = pd.concat([
    selected_cities.nlargest(5, 'DataPoints'),
    selected_cities.loc[[selected_cities['Latitude'].abs().idxmax()]]  
])

for _, row in label_cities.iterrows():
    ax.text(
        row['Longitude'] + 2,
        row['Latitude'],
        row['City'],
        transform=ccrs.PlateCarree(),
        fontsize=9,
        ha='left',
        bbox=dict(facecolor='white', alpha=0.7, pad=2, edgecolor='none')
    )

#CLIMATE zones 
for lat in [-66.5, -23.5, 23.5, 66.5]:
    ax.plot([-180, 180], [lat, lat], 
            color='red', 
            linestyle='--', 
            alpha=0.3, 
            transform=ccrs.PlateCarree())


cbar = plt.colorbar(scatter, orientation='horizontal', pad=0.02, aspect=50)
cbar.set_label('Latitude (Climate Zones)')
cbar.set_ticks([0, 0.25, 0.5, 0.75, 1])
cbar.set_ticklabels(['Polar (90°S)', 'Temperate', 'Tropical', 'Temperate', 'Polar (90°N)'])


plt.title('Geographic Distribution of Selected 30 Cities\n'
         'Color-coded by Latitude with Climate Zone Boundaries',
         pad=20, fontsize=14)
plt.tight_layout()

# Save high-res version

plt.show()

# %%
print(df_selected.head())

# %%

# df_imputed = df_imputed.sort_values('dt')


# split_idx = int(len(df_imputed) * 0.8)
# train = df_imputed.iloc[:split_idx]
# test = df_imputed.iloc[split_idx:]

# print(f"Train: {train['dt'].min().date()} to {train['dt'].max().date()}") 
# print(f"Test: {test['dt'].min().date()} to {test['dt'].max().date()}")
# print(f"Train %: {len(train)/len(df_selected):.1%}, Test %: {len(test)/len(df_imputed):.1%}")

# %% [markdown]
# # CONVERTING THE COORDS TO FLOATS, N/S/E/W -> FLOATS, CYCLIC ENCODING 

# %%
def convert_coordinate(coord):
    if isinstance(coord, str):
        direction = coord[-1].upper()  
        value = float(coord[:-1])      
        if direction in ['S', 'W']:
            value *= -1  
        return value
    return coord  


df_imputed['Latitude'] = df_imputed['Latitude'].apply(convert_coordinate)
df_imputed['Longitude'] = df_imputed['Longitude'].apply(convert_coordinate)

# %%
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score


df_imputed['dt'] = pd.to_datetime(df_imputed['dt'])


df_imputed['year'] = df_imputed['dt'].dt.year
df_imputed['month'] = df_imputed['dt'].dt.month
df_imputed['day_of_year'] = df_imputed['dt'].dt.dayofyear


df_imputed['month_sin'] = np.sin(2 * np.pi * df_imputed['month'] / 12)
df_imputed['month_cos'] = np.cos(2 * np.pi * df_imputed['month'] / 12)


city_dummies = pd.get_dummies(df_imputed['City'], prefix='city')
df_imputed = pd.concat([df_imputed, city_dummies], axis=1)


city_dummy_cols = [col for col in df_imputed.columns if col.startswith('city_')]


df_imputed = df_imputed.drop(columns=city_dummy_cols)


print("Columns after removing one-hot cities:", df_imputed.columns)


# WITHOUT THE CO2 
# feature_cols = ['Latitude', 'Longitude', 'year', 'month_sin', 'month_cos'] + \
#                [col for col in df_imputed.columns if col.startswith('city_')]
# X = df_imputed[feature_cols]
# y = df_imputed['AverageTemperature']


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# %%
print(df_imputed)

# %%
df_emissions = pd.read_csv('owid-co2-data.csv')
df_emissions = df_emissions[df_emissions['year'] >= 1950]  # Adjust column name if needed


# %% [markdown]
# # COMBINING THE TWO DATASET AS COUNTRY IS OUR JOINT

# %%
valid_countries = df_imputed['Country'].unique() 
df_emissions_filtered = df_emissions[df_emissions['country'].isin(valid_countries)]

# %%
print(valid_countries)
df_emissions_filtered.head()

# %%
print("Columns in emissions dataset:", df_emissions.columns.tolist())



# %%
emission_feat = ['co2', 'co2_per_capita', 'coal_co2', 'country', 'year']
df_emissions_filtered = df_emissions[emission_feat].copy()  

# %%
df_emissions_filtered.head()

# %%

df_emissions_filtered = df_emissions_filtered.rename(columns={'country': 'Country'})
df_emissions_filtered = df_emissions_filtered.rename(columns={'year': 'Year'})

df_imputed['Year'] = pd.to_datetime(df_imputed['dt']).dt.year


df_combined = pd.merge(
    df_imputed,               
    df_emissions_filtered,    
    on=['Country', 'Year'],   
    how='left'                
)

# %%

df_combined['co2'] = df_combined.groupby('Country')['co2'].transform(
    lambda x: x.fillna(x.median())
)

# %%
print("Duplicate rows:", df_combined.duplicated().sum())

# %% [markdown]
# # FILLING MISSING DATA OF THE CO2 DATASET

# %%
for col in ['co2', 'co2_per_capita', 'coal_co2']:

    country_medians = df_combined.groupby('Country')[col].median()
    

    df_combined[col] = df_combined[col].fillna(
        df_combined['Country'].map(country_medians)
    )
    

    global_median = df_combined[col].median()
    df_combined[col] = df_combined[col].fillna(global_median)
    
print("Missing values after imputation:")
print(df_combined[['co2', 'co2_per_capita', 'coal_co2']].isnull().sum())

df_combined = df_combined.drop(columns=['Year'], errors='ignore')  

# %% [markdown]
# # SPLITTING THE DATA INTO TRAINING AND VALIDATION SET 70-30 SPLIT 

# %%
from sklearn.model_selection import train_test_split


feature_cols = [
    'Latitude', 'Longitude', 'year', 'month_sin', 'month_cos',
    'co2', 'co2_per_capita', 'coal_co2'  # Emission features
]


target_col = 'AverageTemperature'

X = df_combined[feature_cols]
y = df_combined[target_col]


X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.3,
    random_state=42,  
    shuffle=True,

)


print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print("\nFeature columns:", X_train.columns.tolist())

# %%

# lr = LinearRegression()
# lr.fit(X_train, y_train)  # Make sure this runs without errors


# print("Coefficients:", lr.coef_)
# print("Intercept:", lr.intercept_)

# %%
# print("NaN values in X_train:", X_train.isnull().sum().sum())
# print("NaN values in y_train:", y_train.isnull().sum())

# %%
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import make_pipeline


# pipe = make_pipeline(
#     SimpleImputer(strategy='median'),  # Handles any remaining NaNs
#     LinearRegression()
# )

# pipe.fit(X_train, y_train)


# final_model = pipe.named_steps['linearregression']
# print("Coefficients:", final_model.coef_)
# print("Intercept:", final_model.intercept_)

# %% [markdown]
# # TESTING THE MODELS PERFORMANCES OF ALL FOUR, BASLEINE LINEAR REGRESSION, RF, NN AND XGBOOST

# %%
#BASELINE MODEL 
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

lr = LinearRegression()
lr.fit(X_train, y_train)

pipe = make_pipeline(
    SimpleImputer(strategy='median'),  # Handles any remaining NaNs
    LinearRegression()
)

pipe.fit(X_train, y_train)

# Train/evaluate
y_pred = pipe.predict(X_test)

# Evaluate
y_pred = lr.predict(X_test)
print(f"Linear Regression:")
print(f"  MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"  R²: {r2_score(y_test, y_pred):.2f}")

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import RandomizedSearchCV

# # Train

# rf_optimized = RandomForestRegressor(
#     n_estimators=300,
#     min_samples_split=10,
#     min_samples_leaf=2,
#     max_features='log2',
#     max_depth=None,  # No limit on tree depth
#     random_state=42,
#     n_jobs=-1  # Use all CPU cores
# )


# rf_optimized.fit(X_train, y_train)


# y_pred_rf = rf_optimized.predict(X_test)


# rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
# r2_rf = r2_score(y_test, y_pred_rf)

# print("Optimized Random Forest Results:")
# print(f"- RMSE: {rmse_rf:.4f}")
# print(f"- R²: {r2_rf:.4f}")

from xgboost import XGBRegressor

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

xgb_model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    early_stopping_rounds=50,
    eval_metric=['rmse', 'mae'],
    random_state=42
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=True
)


y_pred = xgb_model.predict(X_test)
xgb_r2 = r2_score(y_test, y_pred)
print(f"\nXGBoost Test R²: {xgb_r2:.4f}")


results = xgb_model.evals_result()
print(f"Final Validation RMSE: {results['validation_0']['rmse'][-1]:.4f}")
print(f"Final Validation MAE: {results['validation_0']['mae'][-1]:.4f}")

# %%
print("Missing values in X_train:")
print(X_train.isnull().sum())

print("\nMissing values in X_test:")
print(X_test.isnull().sum())

# %%
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


rf_optimized = RandomForestRegressor(
    n_estimators=300,
    min_samples_split=10,
    min_samples_leaf=2,
    max_features='log2',
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
rf_optimized.fit(X_train, y_train)


y_pred_rf = rf_optimized.predict(X_test)


mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)


print("\nOptimized Random Forest Results:")
print(f"- MAE:  {mae_rf:.4f}")  
print(f"- RMSE: {rmse_rf:.4f}")
print(f"- R²:   {r2_rf:.4f}")


import matplotlib.pyplot as plt
importances = rf_optimized.feature_importances_
features = X_train.columns 
sorted_idx = np.argsort(importances)

plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_idx)), importances[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), features[sorted_idx])
plt.title("Random Forest Feature Importance")
plt.xlabel("Importance Score")
plt.tight_layout()
plt.show()

# %%
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np


X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)


model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=400,
    early_stopping_rounds=20,
    eval_metric=['rmse', 'mae'],
    verbosity=0,
    reg_alpha=0.5,  
    reg_lambda=1.0, 
    gamma=0.1,
    max_depth=4,
    learning_rate=0.1,

)




model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False  
)


y_pred = model.predict(X_test)
results = model.evals_result()

print("\n=== Final Metrics ===")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"RMSE: {results['validation_1']['rmse'][-1]:.4f}")  
print(f"MAE: {results['validation_1']['mae'][-1]:.4f}")


plt.figure(figsize=(12, 5))


plt.subplot(1, 2, 1)
plt.plot(results['validation_0']['rmse'], label='Train RMSE')
plt.plot(results['validation_1']['rmse'], label='Test RMSE')
plt.legend()
plt.title('RMSE over Training Rounds')
plt.xlabel('Boosting Round')
plt.ylabel('RMSE')


plt.subplot(1, 2, 2)
plt.plot(results['validation_0']['mae'], label='Train MAE')
plt.plot(results['validation_1']['mae'], label='Test MAE')
plt.legend()
plt.title('MAE over Training Rounds')
plt.xlabel('Boosting Round')
plt.ylabel('MAE')

plt.tight_layout()
plt.show()

importance = model.get_booster().get_score(importance_type='weight')

filtered_importance = {
    k: v for k, v in importance.items() 
    if not k.lower().startswith('city_')
}

sorted_idx = np.argsort(list(importance.values()))[-20:]  # Top 20 features

plt.figure(figsize=(10, 6))
plt.barh(
    range(len(sorted_idx)),
    [list(importance.values())[i] for i in sorted_idx],
    align='center'
)
plt.yticks(
    range(len(sorted_idx)),
    [list(importance.keys())[i] for i in sorted_idx]
)
plt.title('XGBoost Feature Importance (Weight)')
plt.xlabel('Importance Score')
plt.tight_layout()
plt.show()

# %%


# %%
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)

# y_pred = rf.predict(X_test)
# print(f"Random Forest:")
# print(f"  MAE: {mean_absolute_error(y_test, y_pred):.2f}")
# print(f"  R²: {r2_score(y_test, y_pred):.2f}")

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np

# param_grid_rf = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_depth': [None, 10, 20, 30, 50],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2']
# }


# rf = RandomForestRegressor(random_state=42)

# rf_search = RandomizedSearchCV(
#     estimator=rf,
#     param_distributions=param_grid_rf,
#     n_iter=50,  # Number of random combinations to try
#     cv=5,       # 5-fold cross-validation
#     scoring='neg_mean_squared_error',  # Optimize for RMSE
#     verbose=1,
#     random_state=42,
#     n_jobs=-1   # Use all CPU cores
# )


# rf_search.fit(X_train, y_train)


# print("Best Random Forest Params:", rf_search.best_params_)

# %%
# print("Best Random Forest parameters:", rf_search.best_params_)
# print("Best score (negative MSE):", rf_search.best_score_)

# %%
# import pandas as pd
# import numpy as np
# from xgboost import XGBRegressor


# X_train = X_train.astype(np.float32)  
# y_train = y_train.astype(np.float32)


# model = XGBRegressor(
#     objective='reg:squarederror',
#     n_estimators=1000,
#     early_stopping_rounds=50,
#     eval_metric='rmse'
# )


# model.fit(
#     X_train.values, 
#     y_train.values,
#     eval_set=[(X_test.values, y_test.values)],
#     verbose=10
# )


# y_pred = model.predict(X_test.values)
# print("MAE:", mean_absolute_error(y_test, y_pred))


# %%
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l1_l2
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),  
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_regularizer=l1_l2(l2=0.01)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mae', 'mse']  
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-5)
]


early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)


y_pred = model.predict(X_test_scaled).flatten()
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\nNeural Network Results:")
print(f"- MAE: {mae:.4f}")  
print(f"- RMSE: {rmse:.4f}")
print(f"- R²: {r2:.4f}")
print(f"- Stopped at epoch: {len(history.history['loss'])}")


import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('MAE During Training')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train MSE')
plt.plot(history.history['val_loss'], label='Validation MSE')
plt.title('MSE Loss During Training')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

# %%
import matplotlib.pyplot as plt
import numpy as np


models = ['Linear Regression', 'Neural Network', 'Random Forest', 'XGBoost']
mae_scores = [5.4900, 0.8246, 0.7856, 0.7469]  
r2_scores = [0.3600, 0.9836, 0.9837, 0.9855]



fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))


bars = ax1.bar(models, mae_scores, color=['#FF9999','#FFB266','#66B2FF','#99FF99'])
ax1.set_title('Model Error Comparison', pad=20, fontsize=14)
ax1.set_ylabel('Error Metric (MAE/RMSE in original units)', fontsize=12)
ax1.set_ylim(0, max(mae_scores)*1.1)


for bar in bars:
    height = bar.get_height()
    ax1.annotate(f'{height:.2f}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),  
                 textcoords="offset points",
                 ha='center', va='bottom', fontsize=10)


bars = ax2.bar(models, r2_scores, color=['#FF9999','#FFB266','#66B2FF','#99FF99'])
ax2.set_title('Model Explained Variance (R²)', pad=20, fontsize=14)
ax2.set_ylabel('R-squared Score (0-1 scale)', fontsize=12)
ax2.set_ylim(0, 1.05)


for bar in bars:
    height = bar.get_height()
    ax2.annotate(f'{height:.4f}',
                 xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom', fontsize=10)


plt.figtext(0.5, 0.01, 
            '*Neural Network shows RMSE (1.5694) instead of MAE for comparison', 
            ha='center', fontsize=10, color='gray')
plt.tight_layout(pad=3.0)
plt.show()

# %% [markdown]
# # SHAP ANALYSIS MORE IN DEPTH TO HOW AND WHICH FEATURES HAS INFLUENCE 

# %%
import shap
import matplotlib.pyplot as plt


explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_train)


plt.style.use('seaborn')


plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_train, plot_type="bar", show=False)
plt.title("Feature Importance (SHAP Values)", fontsize=14)
plt.tight_layout()
plt.show()


plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_train, show=False)
plt.title("Feature Impact on Temperature", fontsize=14)
plt.tight_layout()
plt.show()


for feature in X_train.columns[:6]:  
    plt.figure(figsize=(8, 4))
    shap.dependence_plot(
        feature, 
        shap_values.values, 
        X_train, 
        interaction_index=None,
        show=False
    )
    plt.title(f"Dependence Plot: {feature}", fontsize=12)
    plt.tight_layout()
    plt.show()


sample_idx = 0


shap.plots.force(
    base_value=explainer.expected_value,
    shap_values=shap_values[sample_idx].values,
    features=X_train.iloc[sample_idx],
    matplotlib=True,
    text_rotation=15
)


features_to_show = ["Latitude", "month_cos", "coal_co2", "month_sin", "Longitude"]  # Example


feature_indices = [X_train.columns.get_loc(f) for f in features_to_show]


filtered_shap = shap_values[sample_idx].values[feature_indices]
filtered_features = X_train.iloc[sample_idx, feature_indices]


shap.plots.force(
    base_value=explainer.expected_value,
    shap_values=filtered_shap,
    features=filtered_features,
    feature_names=features_to_show, 
    matplotlib=True,
    text_rotation=15
)

# %%


# %%


# %%


# %%


# %%


# %%



